In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import sys
sys.path.insert(1, 'C:/Users/pnknv/Documents/GitHub/yadisk_loader')
from main_utils import *
from dadata import Dadata

In [3]:
st_time = datetime.now()
print('Скрипт запущен: ', get_today_date())

Скрипт запущен:  20/04/2023, 23:39:11


In [4]:
# для обращения к препрод базе

In [5]:
def get_config_1(get_only_start_time=False):
    config = configparser.ConfigParser()
    config.read('config.ini')

    ssh_host = config['database']['ssh_host_main']  # переключить на ssh_host_main для работы на прод сервере
    ssh_port = int(config['database']['ssh_port'])
    ssh_username = config['database']['ssh_username']
    ssh_password = config['database']['ssh_password']
    database_username = config['database']['database_username']
    database_password = config['database']['database_password']
    database_name = config['database']['database_name']
    preprod_database_name = config['database']['preprod_database_name']
    localhost = config['database']['localhost']
    localhost_port = int(config['database']['localhost_port'])
    table_name = config['database']['table_name']

    ya_token = config['yandex']['ya_token']
    ya_api = config['yandex']['ya_api']
    ya_link = config['yandex']['ya_link_test']

    start_time = int(config['start_time']['daily_start_hour'])

    if get_only_start_time == False:
        return ssh_host, ssh_port, ssh_username, ssh_password, database_username, database_password, preprod_database_name, \
               localhost, localhost_port, table_name, ya_token, ya_api, ya_link
    else:
        return start_time

# чтение локальных файлов

In [90]:
ilya_c = pd.read_csv('циан 16-04-23 (без дублей) new.csv', delimiter=',', encoding='utf8')

In [91]:
ilya_c.head(5)

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Адрес,Дата публикации,Тип продавца,Продавец
0,https://kazan.cian.ru/sale/flat/286179179/,"1-комн. кв., 44,7 м², 1/10 этаж",1-комн. кв.,"44,7 м²",1/10 этаж,7700000,172260,NaN,Республика Татарстан;Казань;р-н Кировский;мкр....,"вчера (вчера, 14:40)",Агентство недвижимости,Альтера
1,https://kazan.cian.ru/sale/flat/286178456/,"1-комн. кв., 43,2 м², 3/24 этаж",1-комн. кв.,"43,2 м²",3/24 этаж,7125000,164931,ЖК «Ракета»,Республика Татарстан;Казань;р-н Советский;улиц...,"два дня назад (13 апр, 14:17)",Риелтор,ID 67064013
2,https://kazan.cian.ru/sale/flat/286176086/,"1-комн. кв., 40,8 м², 4/19 этаж",1-комн. кв.,"40,8 м²",4/19 этаж,7500000,183824,ЖК «Золотая середина»,Республика Татарстан;Казань;р-н Ново-Савиновск...,"вчера (14 апр, 10:54)",Агентство недвижимости,Альтера
3,https://kazan.cian.ru/sale/flat/286172402/,"2-комн. кв., 60 м², 4/7 этаж",2-комн. кв.,60 м²,4/7 этаж,7599000,126650,NaN,Республика Татарстан;Казань;р-н Приволжский;мк...,"1 минуту назад (сегодня, 10:29)",Агентство недвижимости,Ключ
4,https://kazan.cian.ru/sale/flat/286167953/,"2-комн. кв., 45 м², 5/12 этаж",2-комн. кв.,45 м²,5/12 этаж,7500000,166667,ЖК «ул. Максимова»,Республика Татарстан;Казань;р-н Авиастроительн...,"два дня назад (13 апр, 11:34)",Агентство недвижимости,ID 63036280


In [92]:
ilya_a = pd.read_csv('16-04-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [93]:
ilya_a.head(5)

,Ссылка,Заголовок,Комнат,Площадь,Этаж,Цена,Цена за м2,Название жк,Район,Улица,Дата публикации,Тип продавца,Продавец,Ссылка на продавца,Завершённых объявлений
0,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 45 м², 5/5 эт.",2-к. квартира,45 м²,5/5 эт.,3500000,77778.0,NaN,р-н Октябрьский,"ул. Мичурина, 61Б",4 дня назад,Агентство,Городской центр недвижимости,https://www.avito.ru/user/95e1bbf88375e37753b4...,190
1,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,"3-к. квартира, 62,3 м², 1/17 эт.",3-к. квартира,"62,3 м²",1/17 эт.,4980000,79936.0,NaN,NaN,"ул. Титова , д. 12",1 день назад,Застройщик,ОАО ТДСК,https://www.avito.ru/user/aa5496618ee23df30923...,500
2,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,"2-к. квартира, 61,3 м², 6/6 эт.",2-к. квартира,"61,3 м²",6/6 эт.,6009360,98032.0,NaN,NaN,"ул. Василия Кандинского, д. 8",1 день назад,Застройщик,"""СЗ""Карьероуправление""",https://www.avito.ru/user/4187eee51af74e5107d1...,129
3,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 34,9 м², 7/9 эт.",1-к. квартира,"34,9 м²",7/9 эт.,3300000,94556.0,NaN,р-н Советский,"Сибирская ул., 104",6 часов назад,Агентство,Перспектива24-Томск,https://www.avito.ru/user/b1f3c879b3fd07d3d782...,NaN
4,https://www.avito.ru/tomsk/kvartiry/1-k._kvart...,"1-к. квартира, 32 м², 9/9 эт.",1-к. квартира,32 м²,9/9 эт.,2430000,75938.0,NaN,р-н Октябрьский,"ул. Мичурина, 6А",4 дня назад,Частное лицо,NaN,NaN,NaN


# процесс импорта

In [6]:
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config()

In [7]:
sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

In [66]:
def download_yadisk_files_1(yandex_api, sharing_link, handled_files, save_dir, marketplace):
    direct_link = get_direct_link(yandex_api, sharing_link)
    try:
        saved_files = []
        if direct_link:
            download = requests.get(direct_link)
            zips = zipfile.ZipFile(io.BytesIO(download.content))
            cnt = 0
            for member in zips.namelist():
                filename = os.path.basename(member)
                if (not filename or Path(filename).stem in handled_files):
                    continue
                elif 'циан' in filename if marketplace == 'avito' else 'циан' not in filename:
                    continue
                print(member)
                write_saved_file_names_1(Path(member).stem, marketplace)
                src = zips.open(member)
                target = open(os.path.join(save_dir, filename), 'wb')
                with src, target:
                    shutil.copyfileobj(src, target)
                    cnt += 1
                    saved_files.append(filename)
                target.close()
            print('Успешно загружено {} файлов с диска {}: {}'.format(cnt, sharing_link, saved_files))
            is_error = False
            return saved_files, is_error
        else:
            print('Failed to download files from "{}"'.format(sharing_link))
            saved_files = []
            is_error = True
            return saved_files, is_error
    except Exception:
        print(traceback.format_exc())
        saved_files = []
        is_error = True
        return saved_files, is_error

In [12]:
local_save_dir_avito = create_load_save_dir('avito')
local_save_dir_cian = create_load_save_dir('cian')

In [82]:
handled_files_avito = get_saved_files_names('avito')
handled_files_cian = get_saved_files_names('cian')

In [83]:
handled_files_cian

['С†РёР°РЅ 15-04-23 (Р±РµР· РґСѓР±Р»РµР№) new',
 'С†РёР°РЅ 16-04-23 (Р±РµР· РґСѓР±Р»РµР№) new',
 'С†РёР°РЅ 17-04-23 (Р±РµР· РґСѓР±Р»РµР№) new',
 'С†РёР°РЅ 18-04-23 (Р±РµР· РґСѓР±Р»РµР№)',
 'С†РёР°РЅ 19-04-23 (Р±РµР· РґСѓР±Р»РµР№)']

In [10]:
# files_to_process_avito = ['16-04-23 (без дублей).csv']
# files_to_process_cian = ['циан 16-04-23 (без дублей) new.csv']

### cian

In [13]:
# check cian from ilya
for filename in files_to_process_cian:
    # обработка realty циан
    print('обработка файла {} для cian'.format(filename))
    df_cian_realty, file_date, error_file_processing = process_realty(local_save_dir_cian, filename,
                                                                      sql_engine, 'cian')

обработка файла циан 16-04-23 (без дублей) new.csv для cian
добавлен новый район "кварталы 68" в districts функцией district_from_rn_mkrn
добавлен новый район "кварталы 38" в districts функцией district_from_rn_mkrn
добавлен новый район "кварталы 25" в districts функцией district_from_rn_mkrn
Успешно обработан файл циан 16-04-23 (без дублей) new.csv для realty


In [14]:
df_cian_realty.head(2)

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,price,price_sqrm
0,2,286179179,12.0,179,3,Республика Татарстан; Казань; р-н Кировский; м...,44.7,1,10,https://kazan.cian.ru/sale/flat/286179179/,2023-04-16,0,292,Агентство недвижимости,1,7700000,172260
1,2,286178456,12.0,362,3,Республика Татарстан; Казань; р-н Советский; у...,43.2,3,24,https://kazan.cian.ru/sale/flat/286178456/,2023-04-16,0,292,Риелтор,1,7125000,164931


In [15]:
exist_ad_id_cian, error_getting_ad_id = get_exist_ad_id(sql_engine, 'cian')
exist_ad_id_cian = exist_ad_id_cian.ad_id.to_list()

In [16]:
cian_realty_exist = df_cian_realty[df_cian_realty.ad_id.isin(exist_ad_id_cian)][list_realty_cols]
cian_realty_new = df_cian_realty[~df_cian_realty.ad_id.isin(exist_ad_id_cian)][list_realty_cols]

In [17]:
print(len(cian_realty_new))
print(len(cian_realty_exist))
cian_realty_new.head(2)

80582
0


,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new
0,2,286179179,12.0,179,3,Республика Татарстан; Казань; р-н Кировский; м...,44.7,1,10,https://kazan.cian.ru/sale/flat/286179179/,2023-04-16,0,292,Агентство недвижимости,1
1,2,286178456,12.0,362,3,Республика Татарстан; Казань; р-н Советский; у...,43.2,3,24,https://kazan.cian.ru/sale/flat/286178456/,2023-04-16,0,292,Риелтор,1


In [18]:
c_1day = pd.read_csv("циан 15-04-23 (без дублей) new.csv", delimiter=',', encoding='utf8')
c_1m = pd.read_csv("циан 16-03-23 (без дублей).csv", delimiter=';', encoding='utf8')
c_1day['ad_id'] = c_1day['Ссылка'].apply(lambda x: x.split('/')[-2])
c_1m['ad_id'] = c_1m['Ссылка'].apply(lambda x: x.split('/')[-2])
c_1day_adids = c_1day.ad_id.unique().tolist()
c_1m_adids = c_1m.ad_id.unique().tolist()
print(len(c_1day_adids))
print(len(c_1m_adids))

81784
64130


In [19]:
print(len(df_cian_realty[df_cian_realty.ad_id.isin(c_1day_adids)][list_realty_cols]))
print(len(df_cian_realty[~df_cian_realty.ad_id.isin(c_1day_adids)][list_realty_cols]))
print(len(df_cian_realty[df_cian_realty.ad_id.isin(c_1m_adids)][list_realty_cols]))
print(len(df_cian_realty[~df_cian_realty.ad_id.isin(c_1m_adids)][list_realty_cols]))

77680
2902
36167
44415


In [20]:
df_cian_realty.to_csv('processed_cian_16.csv')

### avito

In [21]:
# check avito from ilya
for filename in files_to_process_avito:
    # обработка realty циан
    print('обработка файла {} для avito'.format(filename))
    df_avito_realty, file_date, error_file_processing = process_realty(local_save_dir_avito, filename,
                                                                      sql_engine, 'avito')

обработка файла 16-04-23 (без дублей).csv для avito
Успешно обработан файл 16-04-23 (без дублей).csv для realty


In [22]:
print(len(df_avito_realty))
df_avito_realty.head(2)

120188


,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2
0,3,2888300632,17,391,4,Томск; р-н Октябрьский; ул. Мичурина; 61Б,45.0,5,5,https://www.avito.ru/tomsk/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,3500000.0,77778.0
1,3,2882533936,17,None,5,Томск; ул. Титова ; д. 12,62.3,1,17,https://www.avito.ru/tomsk/kvartiry/3-k._kvart...,2023-04-16,0,1,Застройщик,1,4980000.0,79936.0


In [23]:
a_1day = pd.read_csv("15-04-23 (без дублей).csv", delimiter=';', encoding='utf8')
a_1m = pd.read_csv("16-03-23 (без дублей).csv", delimiter=';', encoding='utf8')
a_1day['ad_id'] = a_1day['Ссылка'].apply(lambda x: x.split('_')[-1])
a_1m['ad_id'] = a_1m['Ссылка'].apply(lambda x: x.split('_')[-1])
a_1day_adids = a_1day.ad_id.unique().tolist()
a_1m_adids = a_1m.ad_id.unique().tolist()
print(len(a_1day_adids))
print(len(a_1m_adids))

121829
121053


In [24]:
print(len(df_avito_realty[df_avito_realty.ad_id.isin(a_1day_adids)][list_realty_cols]))
print(len(df_avito_realty[~df_avito_realty.ad_id.isin(a_1day_adids)][list_realty_cols]))
print(len(df_avito_realty[df_avito_realty.ad_id.isin(a_1m_adids)][list_realty_cols]))
print(len(df_avito_realty[~df_avito_realty.ad_id.isin(a_1m_adids)][list_realty_cols]))

114448
5740
75569
44619


In [25]:
df_avito_realty.to_csv('processed_avito_16.csv')

# новые скрипты для выката

In [312]:
# запросы на добавить поля
# ad_id в dadata_houses
# ALTER TABLE dadata_houses add COLUMN ad_id BIGINT DEFAULT NULL;

In [308]:
# чтение
# df_avito_realty = pd.read_csv('processed_avito_16.csv', index_col=0)
# df_cian_realty = pd.read_csv('processed_cian_16.csv', index_col=0)

In [309]:
# exist_ad_id_cian, error_getting_ad_id = get_exist_ad_id(sql_engine, 'cian')
# exist_ad_id_cian = exist_ad_id_cian.ad_id.to_list()
# df_cian_realty_exist = df_cian_realty[df_cian_realty.ad_id.isin(exist_ad_id_cian)]
# df_cian_realty_new = df_cian_realty[~df_cian_realty.ad_id.isin(exist_ad_id_cian)]

In [310]:
# exist_ad_id_avito, error_getting_ad_id = get_exist_ad_id(sql_engine, 'avito')
# exist_ad_id_avito = exist_ad_id_cian.ad_id.to_list()
# df_avito_realty_exist = df_avito_realty[df_avito_realty.ad_id.isin(exist_ad_id_avito)]
# df_avito_realty_new = df_avito_realty[~df_avito_realty.ad_id.isin(exist_ad_id_avito)]

In [313]:
# realty_exist - сразу в таблицу
# realty_new - в дадату

In [314]:
# запрос к dadata

# def filter_addr_for_dadata(addrString, source):
#     addrList = addrString.split('; ')
#     min_len = 3 if source == 'avito' else 4
#     flag_less = len(addrList) < min_len
#     flag_notHouse = len(addrList[-1]) > 10
#     flag_notHouseNum = not any(del_el in addrList[-1] for del_el in [str(n) for n in range(0,10)])
#     if flag_less or flag_notHouse or flag_notHouseNum:
#         return None
#     else:
#         if source == 'cian':
#             addrList = list(filter(lambda c: 'мкр' not in c, addrList))
#         else:
#             addrList = list(filter(lambda c: not any(del_el in c for del_el in [str(i) for i in range(0,10)] if 'мкр' in c), addrList))
#             addrList = list(filter(lambda c: 'мкр' in c, addrList))
#         return '; '.join(addrList)

# def dadata_request(df, source):
#     st_time = datetime.now()
#     # dadata_credentials
#     token = "f288b25edb6d05b5ceb4d957376104a181c4adee"
#     secret = "9d337ae6b9901a6708802eaca6d7055ce2c64772"
#     dadata = Dadata(token, secret)
#     # create df for dadata_house_loading
#     dh_df = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon', 'street', 'house', 'qc', 'result', 'qc_geo', 'ad_id'])
#     # create var for bad addr count
#     bad_addr = 0
#     # dadata request
#     if source == 'avito':
#         for i, row in df.iterrows():
#             addr = filter_addr_for_dadata(row.addr, source)
#             if addr:
#                 d_res = dadata.clean("address", addr)
#                 dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
#                                                d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
#             else:
#                 bad_addr += 1
#     else: # source == 'cian'
#         for i, row in df.iterrows():
#             addr = filter_addr_for_dadata(row.addr, source)
#             if addr:
#                 d_res = dadata.clean("address", addr)
#                 dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
#                                                d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
#             else:
#                 bad_addr += 1
#     print('обращение к дадата по {}/{} записям из {} заняло'.format(len(df) - bad_addr, len(df), source), datetime.now() - st_time)
#     return dh_df

In [ ]:
# получение данных район, house_id, jkh_id, dadata_house_id

# def get_districts_from_house(df, engine):
#     unique_fias = tuple(df.dropna(subset='house_fias_id').house_fias_id.unique())
#     get_districts_query = \
#         """select dh.house_fias_id, h.district_id, h.id as house_id, h.jkh_id, dh.id as dadata_house_id
#             from dadata_houses dh left join houses h on h.dadata_houses_id = dh.id 
#             where h.district_id is not null
#             and dh.house_fias_id in {}""".format(unique_fias)
#     try:
#         con_obj = engine.connect()
#         districts_db = pd.read_sql(text(get_districts_query), con=con_obj)
#         con_obj.close()
#         exc_code = None
#     except Exception as exc:
#         print(traceback.format_exc())
#         districts_db = None
#         exc_code = exc.code
#     return districts_db, exc_code

In [ ]:
# avito

In [316]:

# dh table
# df_dadata_houses_avito = dadata_request(df_avito_realty_sample, 'avito')
# df_dadata_houses_avito.to_sql(name='dadata_houses', con=sql_engine, if_exists='append', chunksize=5000, method='multi', index=False)

# distr


In [315]:
df_dadata_houses_avito

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.6737522,49.2170794,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672
2,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.1467882,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264
3,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.95511,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396
4,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2412284578
...,...,...,...,...,...,...,...,...,...,...
467,None,{'source': 'Краснодар; р-н Прикубанский; ул. З...,45.087523,38.902657,Западный Обход,52,0,"г Краснодар, ул Западный Обход, стр 52",2,2456149400
468,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.06929,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715
469,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521
470,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916


# обновление функционала для обращения к dadata и записи в иные таблицы

In [146]:
# запросы на добавить поля
# ad_id в dadata_houses
# ALTER TABLE dadata_houses add COLUMN ad_id BIGINT DEFAULT NULL;

In [147]:
# ЦИАН:
# скачал ссв
# обработал до реалти
# фильтр по ад_ид
# те, что есть - только апдейт
## те, что нет - запрос к дадата
##  
##
##
#
#
## скачал ссв

In [11]:
# чтение
df_avito_realty = pd.read_csv('processed_avito_16.csv', index_col=0)
df_cian_realty = pd.read_csv('processed_cian_16.csv', index_col=0)

In [73]:
df_avito_realty[df_avito_realty.city_id == 7]

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2
6449,3,2880852955,7,358,4,Челябинск; р-н Калининский; ул. Университетска...,52.2,7,20,https://www.avito.ru/chelyabinsk/kvartiry/2-k....,2023-04-16,0,1,Частное лицо,1,4300000.0,82375.0
6450,3,2963353542,7,358,4,Челябинск; р-н Калининский; Набережная ул.; 9А,54.0,3,10,https://www.avito.ru/chelyabinsk/kvartiry/2-k....,2023-04-16,0,1,Агентство,1,3850000.0,71296.0
6451,3,3037213229,7,352,4,Челябинск; р-н Металлургический; Трудовая ул.;...,44.2,1,5,https://www.avito.ru/chelyabinsk/kvartiry/2-k....,2023-04-16,0,1,Частное лицо,1,1900000.0,42986.0
6452,3,2874505322,7,351,4,Челябинск; р-н Центральный; пр-т Ленина; 36,41.2,5,5,https://www.avito.ru/chelyabinsk/kvartiry/2-k....,2023-04-16,0,1,Агентство,1,3590000.0,87136.0
6453,3,2982284759,7,349,4,Челябинск; р-н Курчатовский; ул. Профессора Бл...,43.0,10,10,https://www.avito.ru/chelyabinsk/kvartiry/2-k....,2023-04-16,0,1,Частное лицо,1,3000000.0,69767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116720,3,2684720112,7,NaN,2,Челябинск; пос. Пригородный; ул. Ласковая; д. 19,24.3,9,10,https://www.avito.ru/chelyabinsk/kvartiry/kvar...,2023-04-16,0,1,Застройщик,1,2517500.0,103601.0
116721,3,2684213822,7,NaN,2,Челябинск; пос. Пригородный; ул. Ласковая; д. 19,24.1,4,10,https://www.avito.ru/chelyabinsk/kvartiry/kvar...,2023-04-16,0,1,Застройщик,1,2470000.0,102490.0
116722,3,2684501315,7,NaN,2,Челябинск; пос. Пригородный; ул. Ласковая; д. 19,24.3,2,10,https://www.avito.ru/chelyabinsk/kvartiry/kvar...,2023-04-16,0,1,Застройщик,1,2470000.0,101646.0
118409,3,2856130936,7,349,5,Челябинск; р-н Курчатовский; пр-т Краснопольск...,67.8,9,10,https://www.avito.ru/chelyabinsk/kvartiry/3-k....,2023-04-16,0,1,Застройщик,1,4691820.0,69201.0


In [149]:
exist_ad_id_cian, error_getting_ad_id = get_exist_ad_id(sql_engine, 'cian')
exist_ad_id_cian = exist_ad_id_cian.ad_id.to_list()

In [12]:
# 500 случайных со стейтом 111
df_avito_realty_sample = df_avito_realty.sample(500, random_state=111)
df_cian_realty_sample = df_cian_realty.sample(500, random_state=111)

In [297]:
def filter_addr_for_dadata(addrString, source):
    addrList = addrString.split('; ')
    min_len = 3 if source == 'avito' else 4
    flag_less = len(addrList) < min_len
    flag_notHouse = len(addrList[-1]) > 10
    flag_notHouseNum = not any(del_el in addrList[-1] for del_el in [str(n) for n in range(0,10)])
    if flag_less or flag_notHouse or flag_notHouseNum:
        return None
    else:
        if source == 'cian':
            addrList = list(filter(lambda c: 'мкр' not in c, addrList))
        else:
            addrList = list(filter(lambda c: not any(del_el in c for del_el in [str(i) for i in range(0,10)] if 'мкр' in c), addrList))
            addrList = list(filter(lambda c: 'мкр' in c, addrList))
        return '; '.join(addrList)

In [295]:
def dadata_request(df, source):
    st_time = datetime.now()
    # dadata_credentials
    token = "f288b25edb6d05b5ceb4d957376104a181c4adee"
    secret = "9d337ae6b9901a6708802eaca6d7055ce2c64772"
    dadata = Dadata(token, secret)
    # create df for dadata_house_loading
    dh_df = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon', 'street', 'house', 'qc', 'result', 'qc_geo', 'ad_id'])
    # create var for bad addr count
    bad_addr = 0
    # dadata request
    if source == 'avito':
        for i, row in df.iterrows():
            addr = filter_addr_for_dadata(row.addr, source)
            if addr:
                d_res = dadata.clean("address", addr)
                dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
                                               d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
            else:
                bad_addr += 1
    else: # source == 'cian'
        for i, row in df.iterrows():
            addr = filter_addr_for_dadata(row.addr, source)
            if addr:
                d_res = dadata.clean("address", addr)
                dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
                                               d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
            else:
                bad_addr += 1
    print('обращение к дадата по {}/{} записям из {} заняло'.format(len(df) - bad_addr, len(df), source), datetime.now() - st_time)
    return dh_df

In [77]:
token = "f288b25edb6d05b5ceb4d957376104a181c4adee"
secret = "9d337ae6b9901a6708802eaca6d7055ce2c64772"
dadata = Dadata(token, secret)

In [79]:
asdf = dadata.clean("address", 'Владивосток, 2-я поселковая, 3В')

In [80]:
asdf

{'source': 'Владивосток, 2-я поселковая, 3В',
 'result': 'г Владивосток, ул 2-я Поселковая, д 3В',
 'postal_code': '690017',
 'country': 'Россия',
 'country_iso_code': 'RU',
 'federal_district': 'Дальневосточный',
 'region_fias_id': '43909681-d6e1-432d-b61f-ddac393cb5da',
 'region_kladr_id': '2500000000000',
 'region_iso_code': 'RU-PRI',
 'region_with_type': 'Приморский край',
 'region_type': 'край',
 'region_type_full': 'край',
 'region': 'Приморский',
 'area_fias_id': None,
 'area_kladr_id': None,
 'area_with_type': None,
 'area_type': None,
 'area_type_full': None,
 'area': None,
 'city_fias_id': '7b6de6a5-86d0-4735-b11a-499081111af8',
 'city_kladr_id': '2500000100000',
 'city_with_type': 'г Владивосток',
 'city_type': 'г',
 'city_type_full': 'город',
 'city': 'Владивосток',
 'city_area': None,
 'city_district_fias_id': None,
 'city_district_kladr_id': None,
 'city_district_with_type': 'р-н Первомайский',
 'city_district_type': 'р-н',
 'city_district_type_full': 'район',
 'city_dist

In [81]:
str(asdf)

"{'source': 'Владивосток, 2-я поселковая, 3В', 'result': 'г Владивосток, ул 2-я Поселковая, д 3В', 'postal_code': '690017', 'country': 'Россия', 'country_iso_code': 'RU', 'federal_district': 'Дальневосточный', 'region_fias_id': '43909681-d6e1-432d-b61f-ddac393cb5da', 'region_kladr_id': '2500000000000', 'region_iso_code': 'RU-PRI', 'region_with_type': 'Приморский край', 'region_type': 'край', 'region_type_full': 'край', 'region': 'Приморский', 'area_fias_id': None, 'area_kladr_id': None, 'area_with_type': None, 'area_type': None, 'area_type_full': None, 'area': None, 'city_fias_id': '7b6de6a5-86d0-4735-b11a-499081111af8', 'city_kladr_id': '2500000100000', 'city_with_type': 'г Владивосток', 'city_type': 'г', 'city_type_full': 'город', 'city': 'Владивосток', 'city_area': None, 'city_district_fias_id': None, 'city_district_kladr_id': None, 'city_district_with_type': 'р-н Первомайский', 'city_district_type': 'р-н', 'city_district_type_full': 'район', 'city_district': 'Первомайский', 'settle

In [304]:
# df_dadata_houses_avito = dadata_request(df_avito_realty_sample, 'avito')

In [460]:
print(len(df_dadata_houses_avito[~df_dadata_houses_avito.house_fias_id.isna()]))
print(len(df_dadata_houses_avito))

308
472


In [461]:
df_dadata_houses_avito.to_csv('dadata_houses_avito_16_sample.csv')

In [165]:
# авито: 
# запрос к дадате 472 адресов - 3 минуты
# за день приходит 5700 записей по 7 городам
# в итоге время загрузки увеличивается на 5700/500 * 3 = 34 минуты

In [290]:
df_dadata_houses_cian_1 = dadata_request(df_cian_realty_sample, 'cian')

обращение к дадата по 350/500 записям из cian заняло 0:01:55.144500


In [458]:
print(len(df_dadata_houses_cian_1[~df_dadata_houses_cian_1.house_fias_id.isna()]))
print(len(df_dadata_houses_cian_1))

322
350


In [459]:
df_dadata_houses_cian.to_csv('dadata_houses_cian_16_sample.csv')

In [307]:
# циан: 
# запрос к дадате 500 адресов - 2.5 минуты
# за день приходит 2700 записей по 7 городам
# в итоге время загрузки увеличивается на 2902/500 * 3 = 17 минуты

In [257]:
# сейчас в dadata_houses 5000 house_fias_id null на 119000 записей
# у меня 200/500

### загрузка

In [368]:
import json

In [13]:
ddt_avito = pd.read_csv('dadata_houses_avito_16_sample.csv', index_col=0)
ddt_cian = pd.read_csv('dadata_houses_cian_16_sample.csv', index_col=0)

In [14]:
def tojson(x):
    try:
        return json.loads(x.replace("'", '"').replace('None', 'null'))
    except:
        try:
            return json.loads(x.replace('"', "").replace("'", '"').replace('None', 'null'))
        except:
            return None

In [15]:
# ddt_avito['data'] = ddt_avito['data'].apply(lambda x: tojson(x))

In [74]:
a = {1:2}

In [76]:
str(a)

'{1: 2}'

In [16]:
ddt_avito

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.673752,49.217079,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672
2,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264
3,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.955110,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396
4,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2412284578
...,...,...,...,...,...,...,...,...,...,...
467,NaN,{'source': 'Краснодар; р-н Прикубанский; ул. З...,45.087523,38.902657,Западный Обход,52,0,"г Краснодар, ул Западный Обход, стр 52",2,2456149400
468,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.069290,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715
469,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521
470,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916


In [17]:
# connect to preprod db
ssh_host, ssh_port, ssh_username, ssh_password, \
database_username, database_password, database_name, \
localhost, localhost_port, table_name, ya_token, ya_api, ya_link = get_config_1()

sql_server, sql_engine = get_sql_engine(ssh_host, ssh_port, ssh_username, ssh_password, localhost, 
                                        localhost_port, database_username, database_password, database_name)

In [18]:
# ddt_avito.to_sql(name='dadata_houses', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)

In [19]:
# был 125817 в dh

In [20]:
def get_districts_from_house(df, engine):
    unique_fias = tuple(df.dropna(subset='house_fias_id').house_fias_id.unique())
    get_districts_query = \
        """select dh.house_fias_id, dh.ad_id, jh.district_id, h.id as house_id, jh.id as jkh_id, dh.id as dadata_house_id
            from dadata_houses dh 
            left join jkh_houses jh on jh.house_fias_id = dh.house_fias_id 
            left join houses h on h.jkh_id = jh.id
            where h.jkh_id != 0
            and jh.district_id is not null
            and dh.ad_id is not null
            and dh.house_fias_id in {}""".format(unique_fias)
            
    try:
        con_obj = engine.connect()
        districts_db = pd.read_sql(text(get_districts_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        districts_db = None
        exc_code = exc.code
    return districts_db, exc_code

In [82]:
ddt_avito

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.673752,49.217079,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672
2,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264
3,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.955110,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396
4,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2412284578
...,...,...,...,...,...,...,...,...,...,...
467,NaN,{'source': 'Краснодар; р-н Прикубанский; ул. З...,45.087523,38.902657,Западный Обход,52,0,"г Краснодар, ул Западный Обход, стр 52",2,2456149400
468,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.069290,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715
469,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521
470,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916


In [21]:
districts_df, error_loading_districts_from_houses = get_districts_from_house(ddt_avito, sql_engine)
districts_df = districts_df[~districts_df.ad_id.isna()]
districts_df.drop_duplicates(subset=['house_fias_id', 'ad_id', 'district_id'], keep='last', inplace=True)

In [22]:
districts_df

,house_fias_id,ad_id,district_id,house_id,jkh_id,dadata_house_id
0,56cfa35b-7d01-4baa-9fa4-479082f3c6fa,2846867889,40,3353,47571,125843
1,ae8c80b2-0693-4c1c-984e-9c1d05a518d1,2400396509,39,6543,51158,125850
2,71d11cf9-d290-4220-833e-ead7ffc9e56a,2769567588,192,4771,53277,125851
6,3705cd19-b27d-44be-a31b-97c487d51eac,2718806950,34,4897,48724,125871
7,47f566ea-1ac0-45e0-8bcb-5fa3b4905494,2691654617,32,3481,47282,125885
...,...,...,...,...,...,...
168,6c1ba80f-09dd-414e-acdd-1e24d3d2cb19,2580980829,36,73335,52273,126280
169,63c22c91-8def-4c23-8de1-f205f289a57d,3050100040,145,82616,59928,125876
170,bb117766-6bdd-452c-91a9-ab7bab3df15f,2922364689,145,82510,59223,125943
172,16e1329e-315d-41b3-bcc6-d19fff68321f,3018622828,362,83436,57332,126040


In [83]:
df_avito_realty_sample

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2
22574,3,2302244851,12,NaN,3,Казань; с. Усады; Дорожная ул.; 69,41.0,1,3,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,3300000.0,80488.0
5382,3,2160111672,5,NaN,3,Анапа; Анапское ш.; 30,38.5,7,16,https://www.avito.ru/anapa/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,5500000.0,142857.0
92678,3,2709454154,12,NaN,3,Казань; ул. Кул Гали; ЖК «‎Изумрудный»,36.1,3,10,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Застройщик,1,6792440.0,188156.0
32890,3,2954503264,12,360,4,Казань; р-н Ново-Савиновский; ул. Адоратского; 33,55.2,7,9,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,6450000.0,116848.0
46484,3,2982556396,2,unknown_district,3,Краснодар; р-н Прикубанский; жилой массив имен...,32.9,6,18,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Агентство,1,3220000.0,97872.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78617,3,2602021054,2,36,4,Краснодар; р-н Центральный; ул. Обрывная; жилы...,67.0,14,16,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-04-16,0,1,Застройщик,1,12262830.0,183027.0
76145,3,2538472715,2,unknown_district,3,Краснодар; р-н Карасунский; ул. Валерия Гассия...,41.5,3,24,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,7640150.0,184100.0
111705,3,2699999521,2,unknown_district,3,Краснодар; р-н Карасунский; ул. им. Мачуги В.Н...,47.4,21,23,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,7252400.0,153004.0
70338,3,2700786916,2,unknown_district,3,Краснодар; р-н Карасунский; ул. им. Мачуги В.Н...,41.5,23,23,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,6308400.0,152010.0


In [24]:
new_df = df_avito_realty_sample.merge(districts_df[['ad_id', 'house_id', 'jkh_id', 'dadata_house_id']], on='ad_id', how='left')

In [84]:
new_df

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2,house_id,jkh_id,dadata_house_id
0,3,2302244851,12,None,3,Казань; с. Усады; Дорожная ул.; 69,41.0,1,3,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,3300000.0,80488.0,NaN,NaN,NaN
1,3,2160111672,5,None,3,Анапа; Анапское ш.; 30,38.5,7,16,https://www.avito.ru/anapa/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,5500000.0,142857.0,NaN,NaN,NaN
2,3,2709454154,12,None,3,Казань; ул. Кул Гали; ЖК «‎Изумрудный»,36.1,3,10,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Застройщик,1,6792440.0,188156.0,NaN,NaN,NaN
3,3,2954503264,12,360,4,Казань; р-н Ново-Савиновский; ул. Адоратского; 33,55.2,7,9,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,6450000.0,116848.0,26563.0,55461.0,125819.0
4,3,2982556396,2,None,3,Краснодар; р-н Прикубанский; жилой массив имен...,32.9,6,18,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Агентство,1,3220000.0,97872.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,3,2602021054,2,36,4,Краснодар; р-н Центральный; ул. Обрывная; жилы...,67.0,14,16,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-04-16,0,1,Застройщик,1,12262830.0,183027.0,NaN,NaN,NaN
496,3,2538472715,2,None,3,Краснодар; р-н Карасунский; ул. Валерия Гассия...,41.5,3,24,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,7640150.0,184100.0,NaN,NaN,NaN
497,3,2699999521,2,None,3,Краснодар; р-н Карасунский; ул. им. Мачуги В.Н...,47.4,21,23,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,7252400.0,153004.0,NaN,NaN,NaN
498,3,2700786916,2,None,3,Краснодар; р-н Карасунский; ул. им. Мачуги В.Н...,41.5,23,23,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,6308400.0,152010.0,NaN,NaN,NaN


In [26]:
distr_to_update = pd.DataFrame(columns=['jkh_id', 'new_distr'])
for i, val in new_df.iterrows():
    try:
        flagNan = np.isnan(val.district_id)
    except:
        flagNan = val.district_id == None
    if flagNan or val.district_id == 'unknown_district' or val.district_id == None:
        try:
            new_df.at[i, 'district_id'] = districts_df[districts_df.ad_id == val.ad_id].district_id.iloc[0]
        except:
            new_df.at[i, 'district_id'] = None
    else:
        try:
            new_district = districts_df[districts_df.ad_id == val.ad_id].district_id.iloc[0]
            if val.district_id != str(new_district):
                print('distr', val.district_id, 'in current', val.ad_id, 'but', new_district, 'in new_df')
                distr_to_update.loc[len(distr_to_update)] = [int(val.jkh_id), val.district_id]
        except:
            pass

distr 360 in current 2954503264 but 163 in new_df
distr 391 in current 2621846649 but 411 in new_df
distr 350 in current 2943783542 but 71 in new_df
distr 360 in current 2938959321 but 178 in new_df
distr 349 in current 2902193878 but 111 in new_df
distr 391 in current 2685824328 but 412 in new_df
distr 36 in current 2879486710 but 189 in new_df
distr 362 in current 2852105762 but 157 in new_df
distr 352 in current 2779188083 but 62 in new_df
distr 353 in current 2851237898 but 73 in new_df
distr 36 in current 2954281446 but 43 in new_df
distr 358 in current 2818788475 but 108 in new_df
distr 358 in current 2758444149 but 93 in new_df
distr 359 in current 2693751508 but 181 in new_df
distr 36 in current 2946174010 but 189 in new_df


In [27]:
# было
len(new_df[(~new_df.district_id.isna()) & (new_df.house_id.isna())])

35

In [28]:
# стало
len(new_df[~new_df.district_id.isna()])

149

In [29]:
distr_to_update

,jkh_id,new_distr
0,55461,360
1,64985,391
2,41155,350
3,55714,360
4,43968,349
5,64613,391
6,49910,36
7,60054,362
8,42919,352
9,45000,353


In [30]:
tuple(distr_to_update.jkh_id.astype(int).to_list())

(55461,
 64985,
 41155,
 55714,
 43968,
 64613,
 49910,
 60054,
 42919,
 45000,
 52782,
 40892,
 44897,
 55985,
 50536)

In [63]:
def get_index_temp_jkh_houses(engine):
    index_show_query = \
        f"""SHOW indexes FROM temp_jkh_houses"""
    try:
        con_obj = engine.connect()
        index_db = pd.read_sql(text(index_show_query), con=con_obj)
        con_obj.close()
        exc_code = None
    except Exception as exc:
        print(traceback.format_exc())
        index_db = None
        exc_code = exc.code
    return index_db, exc_code

def create_temp_jkh_houses(engine):
    create_table_query = \
        """CREATE TABLE `temp_jkh_houses` (
          `id` int(11) NOT NULL AUTO_INCREMENT,
          `jkh_id` int(11) NOT NULL,
          `new_distr` int(11) DEFAULT NULL,
          PRIMARY KEY (`id`)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8 ROW_FORMAT=DYNAMIC;"""

    index_create_query = \
        f"""CREATE INDEX index_jkh_id_temp
        ON temp_jkh_houses (jkh_id);"""

    clear_temp_table_query = \
        """DELETE FROM temp_jkh_houses"""

    try:
        con_obj = engine.connect()
        con_obj.execute(text(create_table_query))
        if 'index_jkh_id_temp' in get_index_temp_jkh_houses(engine)[0].Key_name.to_list():
            pass
        else:
            con_obj.execute(text(index_create_query))
        con_obj.commit()
        con_obj.close()
        print('Временная таблица temp_jkh_houses создана')
        return None
    except:
        try:
            con_obj = engine.connect()
            con_obj.execute(text(clear_temp_table_query))
            con_obj.commit()
            con_obj.close()
            print('Временная таблица temp_jkh_houses очищена')
            return None
        except:
            return True

def update_jkh_houses(engine, df):
    clear_temp_table_query = \
        """DELETE FROM temp_jkh_houses"""
    try:
        # выгрузка данных в таблицу на сервере
        load_df_into_sql_table(df, 'temp_jkh_houses', engine)

        con_obj = engine.connect()
        common_ids = tuple(df.jkh_id)
        
        update_table_query = f"""update jkh_houses join temp_jkh_houses on jkh_houses.id=temp_jkh_houses.jkh_id
                                            set jkh_houses.district_id = temp_jkh_houses.new_distr
                                            WHERE jkh_houses.id in {common_ids}"""

        con_obj.execute(text(update_table_query))
        con_obj.commit()
        con_obj.close()

#         очистка данных из temp_realty
        con_obj = engine.connect()
        con_obj.execute(text(clear_temp_table_query))
        con_obj.commit()
        con_obj.close()
        return None
    except Exception as exc:
        print(exc)
        try:
            con_obj = engine.connect()
            con_obj.execute(text(clear_temp_table_query))
            con_obj.commit()
            con_obj.close()
            return True
        except:
            print('Не удалось подключиться к БД')
            return True

In [48]:
distr_to_update

,jkh_id,new_distr
0,55461,360
1,64985,391
2,41155,350
3,55714,360
4,43968,349
5,64613,391
6,49910,36
7,60054,362
8,42919,352
9,45000,353


In [71]:
marketplace = 'avito'
filename = 'файл'
('циан' in filename if marketplace == 'avito' else 'циан' not in filename)

False

In [64]:
error_create_temp_jkh_houses = create_temp_jkh_houses(sql_engine)
if error_create_temp_jkh_houses:
    error_create_temp_jkh_houses = True
#     return error_create_temp_jkh_houses, False, False, False
else:
    error_create_temp_realty = False

error_updating_jkh_houses = update_jkh_houses(sql_engine, distr_to_update)
if error_updating_jkh_houses:
    error_updating_jkh_houses = True
#     return False, False, False, error_updating_realty
else:
    error_updating_jkh_houses = False
    print('Обновление данных завершено')

Временная таблица temp_jkh_houses очищена
Обновление данных завершено


In [124]:
# df_test_city.to_sql(name="city", con=sql_engine, if_exists="replace")

1

2023-04-20 17:59:28,733| ERROR   | Socket exception: Удаленный хост принудительно разорвал существующее подключение (10054)


In [ ]:
# ddt_avito.to_sql(name='dadata_houses', con=sql_engine, if_exists='append', chunksize=7000, method='multi', index=False)

In [453]:
new_df = df_avito_realty_sample_1.merge(districts_df[['ad_id', 'house_id', 'jkh_id', 'dadata_house_id']], on='ad_id', how='inner')

In [454]:
new_df

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2,house_id,jkh_id,dadata_house_id
0,3,2954503264,12,360,4,Казань; р-н Ново-Савиновский; ул. Адоратского; 33,55.2,7,9,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,6450000.0,116848.0,12716,55461,125819
1,3,2954503264,12,360,4,Казань; р-н Ново-Савиновский; ул. Адоратского; 33,55.2,7,9,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,6450000.0,116848.0,26563,55461,125819
2,3,2396558525,12,NaN,3,Казань; Республика Татарстан; улица Академика ...,43.9,8,9,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,6200000.0,141230.0,13804,55637,125824
3,3,2396558525,12,NaN,3,Казань; Республика Татарстан; улица Академика ...,43.9,8,9,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,6200000.0,141230.0,80669,55637,125824
4,3,2416219349,4,NaN,3,Новороссийск; с. Цемдолина; ул. Кипарисовая; д. 6,48.6,2,3,https://www.avito.ru/novorossiysk/kvartiry/1-k...,2023-04-16,0,1,Частное лицо,1,5100000.0,104938.0,11068,54025,125831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,3,2266643334,12,NaN,4,Казань; ул. Аделя Кутуя; д. 3,57.9,10,19,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Застройщик,1,9990000.0,172539.0,27224,55383,126279
170,3,2266643334,12,NaN,4,Казань; ул. Аделя Кутуя; д. 3,57.9,10,19,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Застройщик,1,9990000.0,172539.0,27285,55383,126279
171,3,2266643334,12,NaN,4,Казань; ул. Аделя Кутуя; д. 3,57.9,10,19,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Застройщик,1,9990000.0,172539.0,32070,55383,126279
172,3,2580980829,2,36,2,Краснодар; р-н Центральный; Садовая ул.; 159к2,32.6,14,19,https://www.avito.ru/krasnodar/kvartiry/kvarti...,2023-04-16,0,1,Агентство,1,2940000.0,90184.0,4800,52273,126280


In [448]:
new_df = pd.merge(df_avito_realty_sample, districts_df[['ad_id', 'house_id', 'jkh_id', 'dadata_house_id']], how='left', on='ad_id')

In [449]:
new_df

,source_id,ad_id,city_id,district_id,type_id,addr,square,floor,house_floors,link,date,status,version,offer_from,status_new,Цена,Цена за м2,house_id,jkh_id,dadata_house_id
0,3,2302244851,12,NaN,3,Казань; с. Усады; Дорожная ул.; 69,41.0,1,3,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,3300000.0,80488.0,NaN,NaN,NaN
1,3,2160111672,5,NaN,3,Анапа; Анапское ш.; 30,38.5,7,16,https://www.avito.ru/anapa/kvartiry/1-k._kvart...,2023-04-16,0,1,Частное лицо,1,5500000.0,142857.0,NaN,NaN,NaN
2,3,2709454154,12,NaN,3,Казань; ул. Кул Гали; ЖК «‎Изумрудный»,36.1,3,10,https://www.avito.ru/kazan/kvartiry/1-k._kvart...,2023-04-16,0,1,Застройщик,1,6792440.0,188156.0,NaN,NaN,NaN
3,3,2954503264,12,360,4,Казань; р-н Ново-Савиновский; ул. Адоратского; 33,55.2,7,9,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,6450000.0,116848.0,12716.0,55461.0,125819.0
4,3,2954503264,12,360,4,Казань; р-н Ново-Савиновский; ул. Адоратского; 33,55.2,7,9,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,2023-04-16,0,1,Агентство,1,6450000.0,116848.0,26563.0,55461.0,125819.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,3,2602021054,2,36,4,Краснодар; р-н Центральный; ул. Обрывная; жилы...,67.0,14,16,https://www.avito.ru/krasnodar/kvartiry/2-k._k...,2023-04-16,0,1,Застройщик,1,12262830.0,183027.0,NaN,NaN,NaN
556,3,2538472715,2,unknown_district,3,Краснодар; р-н Карасунский; ул. Валерия Гассия...,41.5,3,24,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,7640150.0,184100.0,NaN,NaN,NaN
557,3,2699999521,2,unknown_district,3,Краснодар; р-н Карасунский; ул. им. Мачуги В.Н...,47.4,21,23,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,7252400.0,153004.0,NaN,NaN,NaN
558,3,2700786916,2,unknown_district,3,Краснодар; р-н Карасунский; ул. им. Мачуги В.Н...,41.5,23,23,https://www.avito.ru/krasnodar/kvartiry/1-k._k...,2023-04-16,0,1,Застройщик,1,6308400.0,152010.0,NaN,NaN,NaN


In [346]:
new_df = pd.merge(ddt_avito, districts_df[['house_fias_id','house_id', 'jkh_id', 'dadata_house_id']], how='left', on='house_fias_id')

In [347]:
new_df

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id,house_id,jkh_id,dadata_house_id
0,93dd5940-a82b-4e15-b2f5-e3cbcd65e19e,{'source': 'Казань; с. Усады; Дорожная ул.; 69...,55.673752,49.217079,Дорожная,69,1,"Респ Татарстан, Лаишевский р-н, с Усады, ул До...",2,2302244851,NaN,NaN,NaN
1,c0c52465-8b8c-4d26-a027-f39c15051ba0,"{'source': 'Анапа; Анапское ш.; 30', 'result':...",44.898317,37.354429,Анапское,30,0,"Краснодарский край, г Анапа, ш Анапское, д 30",0,2160111672,NaN,NaN,NaN
2,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264,12716.0,55461.0,40325.0
3,432050a3-99bf-4441-a341-914b1fc79a7d,{'source': 'Краснодар; р-н Прикубанский; жилой...,45.104595,38.955110,Степная,1/1,0,"г Краснодар, тер им. Демьяна Бедного жилой мас...",0,2982556396,NaN,NaN,NaN
4,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2412284578,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,NaN,{'source': 'Краснодар; р-н Прикубанский; ул. З...,45.087523,38.902657,Западный Обход,52,0,"г Краснодар, ул Западный Обход, стр 52",2,2456149400,NaN,NaN,NaN
468,8034091c-17ed-4ffe-86a0-f97336db95d6,{'source': 'Краснодар; р-н Карасунский; ул. Ва...,44.989475,39.069290,им. Валерия Гассия,4/7,1,"г Краснодар, ул им. Валерия Гассия, д 4/7",1,2538472715,75286.0,NaN,106133.0
469,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2699999521,NaN,NaN,NaN
470,4abecbfe-cbe2-4c91-b43b-004f985ea730,{'source': 'Краснодар; р-н Карасунский; ул. им...,45.013931,39.092328,им. Мачуги В.Н.,166,1,"г Краснодар, ул им. Мачуги В.Н., д 166 к 2",1,2700786916,NaN,NaN,NaN


In [322]:
new_df[~new_df.district_id.isna()]

,house_fias_id,data,geo_lat,geo_lon,street,house,qc,result,qc_geo,ad_id,jkh_id,house_id,district_id
3,4ec12aab-20d3-4b7f-bc5e-ee729be0e529,{'source': 'Казань; р-н Ново-Савиновский; ул. ...,55.840791,49.146788,Адоратского,33,0,"г Казань, ул Адоратского, д 33",0,2954503264,55461.0,12716.0,163.0
8,a815b58a-c635-40a1-a125-63f08da387ad,{'source': 'Казань; Республика Татарстан; улиц...,55.759734,49.236589,Академика Завойского,11А,0,"г Казань, ул Академика Завойского, д 11А",0,2396558525,55637.0,13804.0,155.0
16,0eb165f8-b2e3-4c59-9cd1-53581a856293,{'source': 'Новороссийск; с. Цемдолина; ул. Ки...,44.749019,37.710501,Кипарисовая,6,0,"Краснодарский край, г Новороссийск, с Цемдолин...",0,2416219349,54025.0,11068.0,281.0
24,2a0f8076-6db6-4098-96bb-e46fae78b832,{'source': 'Казань; р-н Советский; ул. Анаса Т...,55.794358,49.111497,Анаса Тазетдинова,6,0,"г Казань, ул Анаса Тазетдинова, д 6",4,2619353372,NaN,79961.0,362.0
29,56cfa35b-7d01-4baa-9fa4-479082f3c6fa,{'source': 'Краснодар; р-н Прикубанский; ул. З...,45.091994,38.911231,Западный Обход,42,0,"г Краснодар, ул Западный Обход, д 42 к 8",0,2846867889,47571.0,3353.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,b24ad300-49db-464f-8606-6b90a0b569f0,"{'source': 'Казань; ул. Аделя Кутуя; д. 3', 'r...",55.787277,49.166556,Аделя Кутуя,3,0,"г Казань, ул Аделя Кутуя, д 3",0,2266643334,55383.0,27224.0,362.0
489,6c1ba80f-09dd-414e-acdd-1e24d3d2cb19,{'source': 'Краснодар; р-н Центральный; Садова...,45.041910,38.993926,Садовая,159,3,"г Краснодар, ул Садовая, д 159 к 2",0,2580980829,52273.0,4800.0,36.0
490,dee5ca6a-4c73-4c2b-88a6-9b2a36e01b3e,{'source': 'Краснодар; р-н Карасунский; ул. Ур...,45.036351,39.070530,Уральская,87/7,0,"г Краснодар, ул Уральская, д 87/7",1,2538419583,NaN,75387.0,46.0
493,1c2f294a-1e15-4687-9124-c833fe9ffb11,{'source': 'Казань; Аметьевская магистраль; д....,55.766925,49.171477,Аметьевская магистраль,18,0,"г Казань, ул Аметьевская магистраль, д 18 к 5",1,2719787283,NaN,79908.0,362.0


In [457]:
# поменять district_id через set_index, для этого перезапустить всю хуйню и изменить проставление дистриктов - убрать unknown

In [ ]:
df_avito_realty_sample_1.set_index(['ad_id']).fillna(districts_df.set_index(['common_3', 'common_4'])).reset_index()

In [ ]:
# получилось определить district у 148/5a00

# dump

In [151]:
# def filter_addr_for_dadata(addr, source):
#     # возвращает строку адреса без района и микрорайона (авито) без региона, района, микрорайона (циан)
#     if source == 'avito':
#         return '; '.join([ad_el for ad_el in addr.split('; ') if not any(del_el in ad_el for del_el in ['мкр', 'р-н', 'микрорайон', 'район', 'Район', 'Микрорайон'])])
#     else: # source == 'cian'
#         ad_noreg = '; '.join(addr.split('; ')[1:])
#         return '; '.join([ad_el for ad_el in ad_noreg.split('; ') if not any(del_el in ad_el for del_el in ['мкр', 'р-н', 'микрорайон', 'район', 'Район', 'Микрорайон'])])

# def dadata_request_1(df, source):
#     st_time = datetime.now()
#     # dadata_credentials
#     token = "f288b25edb6d05b5ceb4d957376104a181c4adee"
#     secret = "9d337ae6b9901a6708802eaca6d7055ce2c64772"
#     dadata = Dadata(token, secret)
#     # create df for dadata_house_loading
#     dh_df = pd.DataFrame(columns=['house_fias_id', 'data', 'geo_lat', 'geo_lon', 'street', 'house', 'qc', 'result', 'qc_geo', 'ad_id'])
#     # dadata request
#     if source == 'avito':
#         for i, row in df[:10].iterrows():
#             d_res = dadata.clean("address", filter_addr_for_dadata(row.addr, 'avito'))
#             dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
#                                            d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
#     else: # source == 'cian'
#         for i, row in df[:10].iterrows():
#             d_res = dadata.clean("address", filter_addr_for_dadata(row.addr, 'cian'))
#             dh_df.loc[len(dh_df.index)] = [d_res['house_fias_id'], d_res, d_res['geo_lat'], d_res['geo_lon'], d_res['street'],
#                                            d_res['house'], d_res['qc'], d_res['result'], d_res['qc_geo'], row.ad_id]
#     print('обращение к дадата по {} записям {} заняло'.format(len(df), source), datetime.now() - st_time)
#     return dh_df